In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
import os
import pickle
import numpy as np

In [3]:
batch_size = 32
num_classes = 10
epochs = 200
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
# The data, shuffled and split between train and test sets:
np.random.seed(9)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_complete =np.concatenate((x_train,x_test),axis=0)
y_complete =np.concatenate((y_train,y_test),axis=0)
rand_train = np.random.randint(0,49999,size=10000)
rand_test= np.random.randint(50000,59999,size=2000)
x_train = x_complete[rand_train]
y_train = y_complete[rand_train]
x_test = x_complete[rand_test]
y_test=y_complete[rand_test]
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (10000, 32, 32, 3)
10000 train samples
2000 test samples


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


In [7]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


In [8]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [9]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



Using real-time data augmentation.
Epoch 1/200
312/312 [==============================] - 15s - loss: 2.1072 - acc: 0.2178 - val_loss: 1.8857 - val_acc: 0.3235
Epoch 2/200
312/312 [==============================] - 7s - loss: 1.8312 - acc: 0.3290 - val_loss: 1.7016 - val_acc: 0.3915
Epoch 3/200
312/312 [==============================] - 7s - loss: 1.7141 - acc: 0.3745 - val_loss: 1.5841 - val_acc: 0.4330
Epoch 4/200
312/312 [==============================] - 7s - loss: 1.6409 - acc: 0.3986 - val_loss: 1.5224 - val_acc: 0.4490
Epoch 5/200
312/312 [==============================] - 7s - loss: 1.5864 - acc: 0.4127 - val_loss: 1.4515 - val_acc: 0.4755
Epoch 6/200
312/312 [==============================] - 7s - loss: 1.5420 - acc: 0.4330 - val_loss: 1.3978 - val_acc: 0.5200
Epoch 7/200
312/312 [==============================] - 8s - loss: 1.4999 - acc: 0.4582 - val_loss: 1.3657 - val_acc: 0.5095
Epoch 8/200
312/312 [==============================] - 7s - loss: 1.4798 - acc: 0.4619 - val_los

312/312 [==============================] - 7s - loss: 0.7642 - acc: 0.7330 - val_loss: 0.8651 - val_acc: 0.7030
Epoch 66/200
312/312 [==============================] - 7s - loss: 0.7640 - acc: 0.7300 - val_loss: 0.8617 - val_acc: 0.7015
Epoch 67/200
312/312 [==============================] - 7s - loss: 0.7484 - acc: 0.7403 - val_loss: 0.8105 - val_acc: 0.7240
Epoch 68/200
312/312 [==============================] - 7s - loss: 0.7487 - acc: 0.7425 - val_loss: 0.8191 - val_acc: 0.7240
Epoch 69/200
312/312 [==============================] - 7s - loss: 0.7430 - acc: 0.7420 - val_loss: 0.8232 - val_acc: 0.7130
Epoch 70/200
312/312 [==============================] - 7s - loss: 0.7319 - acc: 0.7428 - val_loss: 0.8204 - val_acc: 0.7130
Epoch 71/200
312/312 [==============================] - 7s - loss: 0.7326 - acc: 0.7438 - val_loss: 0.8311 - val_acc: 0.7200
Epoch 72/200
312/312 [==============================] - 7s - loss: 0.7345 - acc: 0.7375 - val_loss: 0.8252 - val_acc: 0.7185
Epoch 73/200


312/312 [==============================] - 8s - loss: 0.5398 - acc: 0.8150 - val_loss: 0.7536 - val_acc: 0.7440
Epoch 131/200
312/312 [==============================] - 8s - loss: 0.5505 - acc: 0.8137 - val_loss: 0.7826 - val_acc: 0.7520
Epoch 132/200
312/312 [==============================] - 7s - loss: 0.5576 - acc: 0.8142 - val_loss: 0.7781 - val_acc: 0.7445
Epoch 133/200
312/312 [==============================] - 7s - loss: 0.5304 - acc: 0.8165 - val_loss: 0.7698 - val_acc: 0.7495
Epoch 134/200
312/312 [==============================] - 8s - loss: 0.5349 - acc: 0.8178 - val_loss: 0.7698 - val_acc: 0.7400
Epoch 135/200
312/312 [==============================] - 7s - loss: 0.5486 - acc: 0.8129 - val_loss: 0.7692 - val_acc: 0.7375
Epoch 136/200
312/312 [==============================] - 7s - loss: 0.5281 - acc: 0.8188 - val_loss: 0.7558 - val_acc: 0.7390
Epoch 137/200
312/312 [==============================] - 8s - loss: 0.5335 - acc: 0.8186 - val_loss: 0.7429 - val_acc: 0.7390
Epoch 

312/312 [==============================] - 7s - loss: 0.4837 - acc: 0.8337 - val_loss: 0.7539 - val_acc: 0.74457 - acc: 0.834
Epoch 195/200
312/312 [==============================] - 8s - loss: 0.4830 - acc: 0.8344 - val_loss: 0.7511 - val_acc: 0.7490
Epoch 196/200
312/312 [==============================] - 7s - loss: 0.4901 - acc: 0.8377 - val_loss: 0.7846 - val_acc: 0.7505
Epoch 197/200
312/312 [==============================] - 7s - loss: 0.4908 - acc: 0.8375 - val_loss: 0.7822 - val_acc: 0.74504946 - acc: 0.
Epoch 198/200
312/312 [==============================] - 7s - loss: 0.4755 - acc: 0.8411 - val_loss: 0.8038 - val_acc: 0.7480
Epoch 199/200
312/312 [==============================] - 8s - loss: 0.4905 - acc: 0.8376 - val_loss: 0.7780 - val_acc: 0.7405
Epoch 200/200
312/312 [==============================] - 7s - loss: 0.4914 - acc: 0.8408 - val_loss: 0.7443 - val_acc: 0.7430


In [11]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


Saved trained model at C:\Users\Derek\Anaconda3\Scripts\saved_models\keras_cifar10_trained_model.h5 


In [12]:

keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)


In [13]:

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

Model Accuracy = 0.73


In [14]:

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Actual Label = automobile vs. Predicted Label = automobile
Actual Label = horse vs. Predicted Label = deer
Actual Label = bird vs. Predicted Label = frog
Actual Label = airplane vs. Predicted Label = ship
Actual Label = ship vs. Predicted Label = ship
Actual Label = dog vs. Predicted Label = dog
Actual Label = horse vs. Predicted Label = horse
Actual Label = cat vs. Predicted Label = dog
Actual Label = horse vs. Predicted Label = horse
Actual Label = horse vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = horse vs. Predicted Label = horse
Actual Label = horse vs. Predicted Label = horse
Actual Label = truck vs. Predicted Label = truck
Actual Label = dog vs. Predicted Label = bird
Actual Label = deer vs. Predicted Label = deer
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = horse vs. Predicted Label = horse
Actual Label = dog vs. Predicted Label = dog
